In [5]:
import os
import string
from time import time
import random

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize

#stop_words = stopwords.words('english')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]


if __name__ == "__main__":

    run_relevant = True
    main_start = time() 

    articles_df = pd.read_csv("all_articles.csv")

    if run_relevant == True:

        articles = list(articles_df[articles_df['relevant_campbell'] == 1]['text'])

    else:

        articles = list(articles_df[articles_df['relevant_campbell'] == 0]['text'])        
    
    embeddings_dict = {}
    count_dict = {}
    
    for article in articles:
        
        #print(f"Doing embeddings for article {count}")
        
        start = time()
        
        sentences = sent_tokenize(article)
        
        for sentence in sentences:
            words_in_sentence = list(sentence.split(" "))
            for word_ in words_in_sentence:

                word_ = word_.lower()
                word_ = word_.strip()
                word_ = word_.replace(" ", "")
                word_ = word_.replace(",", "")
                word_ = word_.replace(".", "")
                word_ = word_.replace(":", "")

                if lemmatizer.lemmatize(word_) != 'ha' or lemmatizer.lemmatize(word_) != 'wa':
                    word_ = lemmatizer.lemmatize(word_)

                if word_ not in stop_words and word_ not in string.punctuation:
                    if word_ not in embeddings_dict.keys():
                        #embeddings_dict[word_] = model.encode(word_)
                        embeddings_dict[word_] = ""
                        count_dict[word_] = 1

                    else:
                        count_dict[word_] += 1

        count+=1
        #print('Cell took %.2f seconds to run.' % (time() - start))

    #print(sorted(((v, k) for k, v in count_dict.items()), reverse=True))
    output_words = pd.Series(count_dict)
    output_df = pd.DataFrame(pd.Series(count_dict))
    '''
    if run_relevant == True:
        output_df.sort_values(by=[0], ascending=False).to_csv("relevant_words_count.csv")
        
    else:
        output_df.sort_values(by=[0], ascending=False).to_csv("irrelevant_words_count.csv")
    '''
    #print('Script took %.2f minutes to run.' % ((time() - main_start)/60))

In [8]:
output_df = pd.DataFrame(pd.Series(count_dict), columns = ['frequency'])

In [12]:
output_df.reset_index(inplace=True)

In [14]:
output_df = output_df.rename(columns={'index': 'word'})
output_df

word  frequency
0        senator         13
1       reverend          5
2        warnock          9
3     introduces          1
4           bill         33
...          ...        ...
5387   honeymoon          1
5388        pact          1
5389    inspired          1
5390         sea          1
5391    million?          1

[5392 rows x 2 columns]

In [17]:
list(output_df.head(30).word)

['senator',
 'reverend',
 'warnock',
 'introduces',
 'bill',
 'ensure',
 'georgia',
 'car',
 'buyer',
 'automaker',
 'fully',
 'benefit',
 'cost-cutting',
 'tax',
 'credits;',
 'today',
 'introduced',
 'affordable',
 'electric',
 'vehicle',
 'america',
 'act',
 'would',
 'create',
 'phase-in',
 'period',
 '(ev)',
 'sourcing',
 'manufacturing',
 'requirement']

In [37]:
list(output_df.sort_values(by=[0], ascending=False).head(20).index)

Index(['pension', 'said', 'retirement', 'year', 'loan', 'mortgage', 'money',
       'rate', 'student', 'pay', 'ha', 'people', 'plan', 'tax', 'debt', 'also',
       'per', 'say', 'financial', 'new'],
      dtype='object')